# Compare different Berkeley Earth realisations

In [ ]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import xarray as xr

from utils import berkeley, utils

## Difference in global mean

In [ ]:
berkeley_globmean_1850_1900_v2023 = berkeley.read_globmean(ref_period=slice(1850, 1900), version="v2023")
berkeley_globmean_1850_1900_v2024 = berkeley.read_globmean(ref_period=slice(1850, 1900), version="v2024")


In [ ]:
berkeley_globmean_1850_1900_v2023.plot()
berkeley_globmean_1850_1900_v2024.plot()

In [ ]:
berkeley_globmean_abs_v2023 = berkeley.read_globmean(ref_period=None, version="v2023")
berkeley_globmean_abs_v2024 = berkeley.read_globmean(ref_period=None, version="v2024")

In [ ]:
berkeley_globmean_abs_v2023.plot()
berkeley_globmean_abs_v2024.plot()

In [ ]:
diff_anom = berkeley_globmean_1850_1900_v2023 - berkeley_globmean_1850_1900_v2024

diff_abs = berkeley_globmean_abs_v2023 - berkeley_globmean_abs_v2024

f, ax = plt.subplots()

diff_anom.plot(ax=ax, label="v2023 - v2024: anom. temp.")
diff_abs.plot(ax=ax, label="v2023 - v2024: abs. temp.")

plt.legend()

ax.axhline(0, lw=0.5, c="0.1")


## Difference in spatial data

In [ ]:
period = slice("1955", "1960")

tmax_2024 = berkeley.read_full("TMAX", version="v2024")
tmin_2024 = berkeley.read_full("TMIN", version="v2024")

In [ ]:
tmax_2023 = berkeley.read_full("TMAX", version="v2023")
tmin_2023 = berkeley.read_full("TMIN", version="v2023")


In [ ]:
tmax_2023.climatology.load()
tmax_2024.climatology.load()
None

In [ ]:
tmin_2023.climatology.load()
tmin_2024.climatology.load()
None

In [ ]:
tmax_2023

In [ ]:
tmax_2024

### Difference in climatology

In [ ]:
f, ax = plt.subplots()

h = (tmax_2024.climatology - tmax_2023.climatology).mean("day_number").plot(ax=ax, add_colorbar=False)

cbar = plt.colorbar(h, ax=ax)

cbar.set_label("climatology difference (°C)")

ax.set_title("Annual mean difference in TMAX climatology (v2023.11.30 - v2022.09.07)", loc="left", fontsize=8)

# plt.savefig("BerkeleyEarth_TMAX_climatology_diff.png")




In [ ]:
(tmin_2023.climatology - tmin_2024.climatology).mean("day_number").plot()


In [ ]:
lon = 8
lat= 47

# (tmax_2023.climatology - tmax_2024.climatology).sel(lat=lat, lon=lon, method="nearest").plot()

tmin_2023.climatology.sel(lat=lat, lon=lon, method="nearest").plot(label="TMIN - v2023", color="r")
tmax_2023.climatology.sel(lat=lat, lon=lon, method="nearest").plot(label="TMAX - v2023", color="r")


tmin_2024.climatology.sel(lat=lat, lon=lon, method="nearest").plot(label="TMIN - v2024", color="b")
tmax_2024.climatology.sel(lat=lat, lon=lon, method="nearest").plot(label="TMAX - v2024", color="b")


plt.legend()


In [ ]:
a = tmin_2023.climatology.sel(lat=lat, lon=lon, method="nearest")
b = tmax_2023.climatology.sel(lat=lat, lon=lon, method="nearest")

plt.fill_between(a.day_number, a, b, color="#3182bd88", label="2023")


a = tmin_2024.climatology.sel(lat=lat, lon=lon, method="nearest")
b = tmax_2024.climatology.sel(lat=lat, lon=lon, method="nearest")

plt.fill_between(a.day_number, a, b, color="#c51b8a88", label="2024")

plt.legend()

plt.title("TX and TN difference")

### Compare TXx

In [ ]:
tmax_seas_2023 = berkeley.add_climatology(tmax_2023)

In [ ]:
tmax_seas_2024 = berkeley.add_climatology(tmax_2024)

In [ ]:
with xr.set_options(use_flox=True):
    txx_2023 = tmax_seas_2023.groupby("time.year").max("time", engine="flox").compute()

In [ ]:
with xr.set_options(use_flox=True):
    txx_2024 = tmax_seas_2024.groupby("time.year").max("time", engine="flox").compute()

In [ ]:
(txx_2024 - txx_2023).sel(year=2000).plot(robust=True)